<a href="https://colab.research.google.com/github/ieea/Agrani_App/blob/main/Fine_tune_BNS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
%pip install unsloth

In [2]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM
import torch
import os
from peft import PeftModel

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True
quantized_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

In [ ]:
lora_layers_and_quantized_model= FastLanguageModel.get_peft_model(
    quantized_model,
    r=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=True,
)

In [ ]:
from datasets import load_dataset

#Importing the dataset
#dataset = load_dataset('csv', data_files='/content/drive/MyDrive/QA_Public_data.xlsx')
#dataset["train"][0]

In [ ]:
!pip install openpyxl

In [ ]:
!pip install datasets --upgrade   # Upgrade the datasets library to the latest version

In [ ]:
from datasets import load_dataset
from datasets import Dataset
import pandas as pd

# Load the Excel file using pandas
df = pd.read_excel('/content/drive/MyDrive/QA_Public_data.xlsx')

# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df, split='train')

# Assuming your Excel file has columns for 'Question', 'Answer', and 'Type'
# Create a new 'conversations' column in the desired format
def create_conversation(row):
  return {"conversations": [{"from": "human", "value": row["Question"]}, {"from": "assistant", "value": row["Answer"]}]}

dataset = dataset.map(create_conversation, batched=False)

dataset = dataset.rename_column('Type', 'instruction') # optional: rename to align with the chat template function
# Instead of adding a new column, replace the existing 'conversations' column
dataset = dataset.remove_columns(['Question', 'Answer', 'instruction'])
#Now apply the standardize_sharegpt function
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

# Access the data
print(dataset[0])

In [ ]:
print(dataset[1])

In [ ]:
dataset.save_to_disk('/content/drive/MyDrive/dataset/sharegpt_dataset')
